**EEG Classification Model training**


In [ ]:
pip install pyedflib

     |████████████████████████████████| 2.4 MB 4.9 MB/s 


In [ ]:
import pyedflib
import numpy as np
from scipy import signal

**Loading Data From EDF File Format**

In [ ]:
def load_signal_from_edffile(path):
  f= pyedflib.EdfReader(path)
  n = f.signals_in_file
  signal_labels = f.getSignalLabels()
  sigbufs = np.zeros((n, f.getNSamples()[0]))
  for i in np.arange(n):
          sigbufs[i, :] = f.readSignal(i)
  annotations = f.read_annotation()
  f._close()
  return sigbufs.transpose(), annotations

**Preprocessing**

Extracting data by:

1. Fitering the signal between 5 and 45Hz.
2. Extracting data from 0.5s before the action till 1s afterwards

In [ ]:
def preprocessing(x, y):
  b, a = signal.butter(5, 5/80, 'high')
  x = signal.filtfilt(b,a,x)
  b, a = signal.butter(5, 45/80, 'low')
  x = signal.filtfilt(b,a,x)
  data = np.zeros((29, 240, 64))
  for t0 in range(1,30):
    data[t0-1, :, :] = x[int(y[t0][0]/10000000*160-80):int(y[t0][0]/10000000*160+160),:]
  u = data.mean(0).reshape((1,)+data.shape[1:])
  o = np.maximum(data.std(0).reshape((1,)+data.shape[1:]), 1e-10)
  data = (data - u)/o
  return data

**Getting Labels Data**

In [ ]:
def get_label(y):
  label = np.array([])
  for i in range(1,30):
    h = y[i][2].decode('utf-8')
    if h == 'T0':
      label = np.append(label, [0,0,1], axis=0)
    if h == 'T1':
      label = np.append(label, [0,1,0], axis=0)
    if h == 'T2':
      label = np.append(label, [1,0,0], axis=0)
  label = label.reshape(29,3)
  return label

In [ ]:
import scipy.io
daata = scipy.io.loadmat('/content/drive/MyDrive/new_data.mat')
data1 = daata['data']

In [ ]:
import scipy.io
daata = scipy.io.loadmat('/content/drive/MyDrive/labels.mat')
label = daata['labels']

**Extracting Data**

Extracting data only form the electrodes as per the 16-channel OpenBCI Headset

In [ ]:
data = np.zeros((11281,240,16))
for i in range(0,11281):
  data[i,:,0] = data1[i,:,21]
  data[i,:,1] = data1[i,:,23]
  data[i,:,2] = data1[i,:,8]
  data[i,:,3] = data1[i,:,12]
  data[i,:,4] = data1[i,:,46]
  data[i,:,5] = data1[i,:,54]
  data[i,:,6] = data1[i,:,60]
  data[i,:,7] = data1[i,:,62]
  data[i,:,8] = data1[i,:,29]
  data[i,:,9] = data1[i,:,37]
  data[i,:,10] = data1[i,:,31]
  data[i,:,11] = data1[i,:,35]
  data[i,:,12] = data1[i,:,40]
  data[i,:,13] = data1[i,:,41]
  data[i,:,14] = data1[i,:,48]
  data[i,:,15] = data1[i,:,52]
data = (data+1)/2

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import AveragePooling2D
from keras import regularizers

**Making the CNN model**

The model will have dimensions as per the "EEGNet" model.

(Credits for this model go to hauke-d/cnn-eeg)

In [ ]:
nchan = 16
input_shape = (240, 16, 1)
l1 = 0.02
model = Sequential()
model.add(Conv2D(40, (65, 1), activation="relu", kernel_regularizer=regularizers.l1(l1), padding="same", input_shape=input_shape))
model.add(Conv2D(40, (1, nchan), activation="relu", kernel_regularizer=regularizers.l1(l1), padding="valid"))
model.add(AveragePooling2D((25, 1), strides=(15, 1)))
model.add(Flatten())
model.add(Dense(80, activation="relu"))
model.add(Dense(3, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["acc"])

In [ ]:
model.fit(data_subject1.reshape(109, 240, 16,1), label, batch_size=4, epochs=18, validation_split=0.4)

In [ ]:
model.save('Model')

INFO:tensorflow:Assets written to: New_Model/assets
